In [1]:
import pandas as pd
import numpy as np
import statistics
import requests
import json
import sys

## Common Methods 

### API Call

In [2]:
def callApi(method, stock, isSeriese = False):
    key = "token=" + "pk_ee5af08dfb9a419aaba2cbee77b80165"
    host = "https://cloud.iexapis.com/stable/"
    ticker = "stock/"+ stock
    api_method ="/" + method 
    query = "?"
    add = "&"
    last5 = "/5y"
    last10 = "/10"
    periodA = "period=annual"
    period = "period=annual"
    if method == "dividends":
        url = host + ticker + api_method + last5 + query + key
    elif (method == "financials" or method == "balance-sheet" or method == "earnings" or method == "cash-flow") and isSeriese :
        url = host + ticker + api_method + last10 + query + periodA + add + key
    else:
        url = host + ticker + api_method + query + key 
    try:
        response = requests.request("GET", url)
        balance_sheet = json.loads(response.text)
        return balance_sheet
    except ValueError:
        print("Unknown expeption on" + stock)
        return {}

### COMPOUND ANNUAL GROWTH RATES (CAGR)

#### CAGR=(( EV / BV) ^ 1/n ) -1

##### where:
##### EV=Ending value
##### BV=Beginning value
##### n=Number of years

In [3]:

def CAGR(item):
    periods = len(item)
    if periods > 1 :
        first = item[-1]
        last = item[0] 
        cage = ((last/first)**(1/periods))-1
        return cage
    else:
        return 0

#### Future Value

##### FV = PV* (1 + r) ^ n
##### PV The present value
##### r The interest rate 
##### n number of periods 

In [4]:
def futureValue(bookValue, period, rate ):
    return bookValue * ((1 + rate) ** period )

In [5]:
def sharesOutstanding(stats):
    if not("sharesOutstanding" in stats.keys()):
        return 0.0
    if stats["sharesOutstanding"] is not None:
        return float(stats["sharesOutstanding"])
    else:
        return 0.0

In [6]:
def EPS(earnings, earning):
    earnings = addTTMToYearLists("earnings", earnings, earning)

    if not("earnings" in earnings.keys()) or len(earnings["earnings"]) == 0:
        return 0.0
    earningsSet = earnings["earnings"]
    eps = np.array([])
    for earning in earningsSet: 
        actualEPS = float(earning["actualEPS"])
        if actualEPS <= 0:
            eps = np.append(eps, 0.1) 
        else: 
            eps = np.append(eps, actualEPS)
    return eps

In [50]:
def shareholdersEquity(financials, financial):
    financials = addTTMToYearLists("financials", financials, financial)
    if not("financials" in financials.keys()) or len(financials["financials"]) < 0:
        return 0.0
    financial = financials["financials"][1]
    if not("shareholderEquity" in financial.keys()):
        return 0.0
    return financial["shareholderEquity"]

In [8]:
def addTTMToYearLists(method,lists,ttmList):
    reportDate = "reportDate"
    if method == "earnings":
        reportDate = "EPSReportDate"
    if not (method in ttmList.keys()):
        return lists
    first = lists[method][0]
    ttm = ttmList[method][0]
    if first[reportDate] != ttm[reportDate]: 
        lists[method].insert(0, ttmList[method][0])
    return lists

# Book value growth and dividend valuation

In [9]:
def avgDividends(dividend):
    avgDividend = np.array([])
    for dividend in dividends:
        if "amount" in dividend.keys() and dividend["amount"] != '': 
            if dividend["amount"] is not None:
                avgDividend = np.append(avgDividend, float(dividend["amount"]))
    if len(avgDividend) > 0: 
        return statistics.mean(avgDividend) 
    else:  
        return 0 

In [10]:
def bookValues(balancesheets, balancesheet):
    balancesheets = addTTMToYearLists("balancesheet", balancesheets, balancesheet)
    if not("balancesheet" in balancesheets.keys()) or len(balancesheets["balancesheet"]) == 0:
        return []
    balancesheetsData = balancesheets["balancesheet"]
    bookValue = np.array([])
    for bs in balancesheetsData:
        treasuryStock = 0.0
        if bs["treasuryStock"] is not None:
            treasuryStock = bs["treasuryStock"]
           
        bookValue = np.append(bookValue, bs["shareholderEquity"]  / (bs["commonStock"] - treasuryStock))
    return bookValue


In [11]:
def BookValueGrowth_DividendValuation(balancesheets, lastBalancesheet, stats, dividends): 
    np.seterr(all='print')
    # Sharesoutstanding
    shares_outstanding = sharesOutstanding(stats)
    
    #Book Value
    bookValue = bookValues(balancesheets, lastBalancesheet) 
    
    # Current Year Book Value 
    currentYearBookValue = 0
    period = len(bookValue)
    if period >0:
        currentYearBookValue = bookValue[0]
    
    # Book value growth rate
    bookValueGrowthRate = CAGR(bookValue)
    adjustedbookValueGrowthRate = .1
    if bookValueGrowthRate > .05:
        adjustedbookValueGrowthRate =  bookValueGrowthRate
    
    # Future Book Value
    futureBookValue = futureValue(currentYearBookValue, len(bookValue), adjustedbookValueGrowthRate)
    
    # Annual Dividend
    annualDividend = avgDividends(dividends)

    
    # Dicount Rate 10y fed note 
    dicountRate10YearFedNote = .0068

    return annualDividend*((1-(1/(1+ dicountRate10YearFedNote)**period))/dicountRate10YearFedNote)+(futureBookValue/(1+dicountRate10YearFedNote)**period)
 

# Benjamin Grahams Intrinsic Value Calculation 

#### ROE = Net Income / Shareholder Equity

In [12]:
def returnOnEquitySerice(financials, financial):
    financials = addTTMToYearLists("financials", financials, financial)
    if not("financials" in financials.keys()) or len(financials["financials"]) == 0:
        return []
    financialsData = financials["financials"]
    returnOnEquity = np.array([])
    for financial in financialsData:
        if "shareholderEquity" in financial.keys() and "netIncome" in financial.keys() : 
            returnOnEquity = np.append(returnOnEquity, financial["netIncome"]/financial["shareholderEquity"])
    return returnOnEquity

#### Net profit margin = Net income / Revenue

In [13]:

def netProfitMarginSerice(financials, financial):
    financials = addTTMToYearLists("financials", financials, financial)
    #Net profit margin = (Net income/ Revenue)
    if not("financials" in financials.keys()) or len(financials["financials"]) == 0:
        return []
    financialsData = financials["financials"]
    netProfitMargin = np.array([])
    for financial in financialsData:
        if "totalRevenue" in financial.keys() and "netIncome" in financial.keys() : 
            netIncome = financial["netIncome"]
            totalRevenue = financial["totalRevenue"]
            if netIncome is not None and totalRevenue is not None:
                netProfitMargin = np.append(netProfitMargin, float(netIncome)/float(totalRevenue))
    return netProfitMargin

In [48]:
def totalRevenue(financials, financial):
    #financials = addTTMToYearLists("financials", financials, financial)
    #print(financials)
    if not("financials" in financials.keys()) or len(financials["financials"]) < 1 :
        return 0.0
    financial = financials["financials"][1]
    if not("totalRevenue" in financial.keys()):
        return 0.0
    return financial["totalRevenue"]

### Intrinsic value = [EPS × (7 + 1.125 *g) × 4.4]/Y 

#### P/E base for a no-growth company – 7 (Indian version mostly 7)
#### g –Average growth rate is the estimated over years available
#### Y - Current yield on AAA-rated corporate bonds.
#### Prevailing (1962) rate on high-grade corporate bonds listed on the New York Stock Exchange -4.4
#### Growth aggressive value  - 1 ( aggressive can be 2)

In [42]:
def BenjaminGrahamsIntrinsicValue(financials,financial, earnings, earning, stats, price):

    #ROE = Net Income / Shareholder Equity
    roe = returnOnEquitySerice(financials, financial)
    if len(roe) > 0:  
        medianROE = statistics.median(roe)
    else:
        return 0
    
    #Net profit margin = (Net income/ Revenu)
    netProfitMargin = netProfitMarginSerice(financials, financial)
    if len(netProfitMargin) > 0:
        meanNetProfit  = statistics.median(netProfitMargin)
    else: 
        return 0
    
    # EPS
    eps = EPS (earnings, earning)
    # CAGR on EPS
    epsCAGR = CAGR(eps)
   
    # Shareholders equity
    shareholders_equity = shareholdersEquity(financials, financial)
    print(shareholders_equity)
    # Sharesoutstanding
    shares_outstanding = sharesOutstanding(stats)
    print(shares_outstanding)
    # Total Revenue
    total_revenue = totalRevenue(financials, financial)
    print(total_revenue)
    # 20 AA Corporate bond Rate %
    #https://ycharts.com/indicators/us_corporate_aa_effective_yield
    Bond_Yeild_AAA = 4.22
    
    # Current Stock Price
    current_stock_price = price
    
    # Required margin of safety
    margin_of_safety = 30
    
    #************************************#
    # Normalized EPS Using The ROE Method
    
    #Normalized Net Income
    net_income_roe = medianROE * shareholders_equity
    # EPS = net income / share 
    eps_roe = net_income_roe / shares_outstanding
    
    #****************************************#
    #Normalized EPS Using The Net Margin Method
    net_income_on_net_margin = meanNetProfit * total_revenue
    eps_net_income_on_net_margin = net_income_on_net_margin / shares_outstanding
    
    #****************************************#
    #Average EPS 
    average_EPS = statistics.mean([eps_roe, eps_net_income_on_net_margin] )
    #Estimated Future EPS Growth Rate
    future_EPS_growth_rate = epsCAGR
    
    ########################################
    #Intrinsic value = [EPS × (7 + 1 *g ) × 4.4]/Y 
    intrinsicValue = (average_EPS * (7 + 1 * future_EPS_growth_rate) *4.4) / Bond_Yeild_AAA
    if intrinsicValue <= 0:
        return 0
    else: 
        return intrinsicValue

# Guru Focus Intrinsic Value

### Free Cash Flow 	=	Cash Flow from Operations	+	Capital Expenditure

In [16]:
def freeCashFlowSerice(cashflows, cashflow):
    cashflows = addTTMToYearLists("cashflow",cashflows,cashflow)
    if not("cashflow" in cashflows.keys()) or len(cashflows["cashflow"]) == 0:
        return []
    cashflowData = cashflows["cashflow"]
    cashFlowsSerice = np.array([])
    for cf in cashflowData:
        if "cashFlow" in cf.keys() and "capitalExpenditures" in cf.keys(): 
            cashFlowValue = cf["cashFlow"]
            capitalExpenditures = cf["capitalExpenditures"]
            if cashFlowValue is None:
                cashFlowValue = 0
            if capitalExpenditures is None:
                capitalExpenditures = 0
            cashFlowsSerice = np.append(cashFlowsSerice, cashFlowValue + capitalExpenditures)
    return cashFlowsSerice

##### Intrinsic Value=(Growth Multipler * Free Cash Flow (6 year avg) + Total Stockholders Equity (Jun20)*0.8) / Shares Outstanding (Diluted Average)

In [17]:
def GuruFocusIntrinsicValue(financials, financial, cashflows,cashflow, earnings, earning, stats):
    #Intrinsic Value=(Growth Multipler * Free Cash Flow (6 year avg)+Total Stockholders Equity (Jun20)*0.8)/Shares Outstanding (Diluted Average)
    
    # https://www.gurufocus.com/news/97735
    growthMultipler = 9.5203515959648

    #Free Cash Flow 
    free_cash_flow = freeCashFlowSerice(cashflows, cashflow)
    mean_free_cash_flow = statistics.mean(free_cash_flow)
    
    # Total Stockholders Equity
    stockholders_equity = shareholdersEquity(financials, financial)
    
    # Sharesoutstanding
    shares_outstanding = sharesOutstanding(stats)
    if stockholders_equity > 0:
        return (growthMultipler * mean_free_cash_flow + stockholders_equity *0.8)/shares_outstanding
    else: 
        return (growthMultipler * mean_free_cash_flow + stockholders_equity / 0.8)/shares_outstanding

In [55]:
tickers = ['BA']
dataSet = [] 
for ticker in tickers:
    data = {} 
    
    # API calls
    balancesheets = callApi("balance-sheet", ticker, True)
    balancesheet = callApi("balance-sheet", ticker)
    stats = callApi("stats", ticker)
    dividends = callApi("dividends", ticker)
    price = callApi("price", ticker)
    financials = callApi("financials", ticker, True)
    financial = callApi("financials", ticker)
    earnings = callApi("earnings", ticker, True)
    earning = callApi("earnings", ticker)
    cashflows = callApi("cash-flow", ticker, True)
    cashflow = callApi("cash-flow", ticker)
    
    # Common data
    data["Ticker"] = ticker
    data["price"] = price
    
    # Book value growth and dividend valuation

    intrinsicValue =  BookValueGrowth_DividendValuation(balancesheets, balancesheet, stats, dividends)
    if intrinsicValue is not None and price is not None and intrinsicValue != 0:
        marginOfsafety = 1 - (float(price)/ intrinsicValue)
        annualReturn = CAGR([intrinsicValue, float(price)])
        data["Book Value Dividend"] =  {"IV": round(intrinsicValue, 2)},{"MS %": round(marginOfsafety *100,1) },{"AR %": round(annualReturn *100, 1)}
    else:
        data["Book Value Dividend"] = "NA"
    
    
    # Benjamin Grahams Intrinsic Value
    intrinsicValue =  BenjaminGrahamsIntrinsicValue(financials, financial, earnings, earning, stats, price)
    if intrinsicValue is not None and price is not None and intrinsicValue != 0:
        marginOfsafety = 1 - (float(price)/ intrinsicValue)
        annualReturn = CAGR([intrinsicValue, float(price)])
        data["Benjamin Grahams"] =  {"IV": round(intrinsicValue, 2)},{"MS %": round(marginOfsafety *100,1) },{"AR %": round(annualReturn *100, 1)}
    else:
        data["Benjamin Grahams"] = "NA"
    
    # Guru Focus Intrinsic Value
    intrinsicValue =  GuruFocusIntrinsicValue(financials, financial, cashflows, cashflow, earnings, earning, stats)
    if intrinsicValue is not None and price is not None and intrinsicValue != 0:
        marginOfsafety = 1 - (float(price)/ intrinsicValue)
        annualReturn = CAGR([intrinsicValue, float(price)])
        data["Guru Focus"] =  {"IV": round(intrinsicValue, 2)},{"MS %": round(marginOfsafety *100,1) },{"AR %": round(annualReturn *100, 1)}
    else:
        data["Guru Focus"] = "NA"
    dataSet.append(data.copy())
pd.DataFrame(dataSet) 

-8617000000
564449000.0
76559000000


,Ticker,price,Book Value Dividend,Benjamin Grahams,Guru Focus
0,BA,160.23,"({'IV': 9.5}, {'MS %': -1586.3}, {'AR %': -75.6})",NA,"({'IV': 64.6}, {'MS %': -148.0}, {'AR %': -36.5})"
